In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
import tensorflow as tf
import numpy as np
#assert float(tf.__version__[:3]) >= 2.3
import os
import cv2
import pathlib
#import tf.keras.models

In [ ]:
# definimos funcion para cargar imagenes de un dataset representativo
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images): #.batch(1).take(100)
    yield [input_value]

#definimos funcion que cargue las imagenes de un directorio
def load_images_from_folder(folder_path):
    images = []
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            img_path = os.path.join(root, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return images

In [ ]:
folder_path = './representative_dataset'

# Cargar imágenes desde la carpeta
images = load_images_from_folder(folder_path)
# Normalizar los valores de píxeles utilizando el preprocesamiento de Xception
images_preproc = [(image / 127.5) - 1 for image in images]
# Convertir la lista de imágenes a un tensor
train_images = tf.convert_to_tensor(images_preproc)

# Para debuggear que la arquitectura cargada sea la misma.
#model = load_model('/content/drive/MyDrive/Colab Notebooks/Xception_model.h5')
#model.summary()

# Cargar el modelo Keras desde el archivo HDF5
keras_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Xception_model.h5')
# Imprimir el tipo del modelo
print(type(keras_model))

# Crear un convertidor TFLite y convertir el modelo
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
#tflite_model = converter.convert()

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

tflite_model_quant_file = "/content/drive/MyDrive/Colab Notebooks/xception_quant.tflite"
#tflite_model_quant_file.write_bytes(tflite_quant_model)
# Escribir los bytes en el archivo
with open(tflite_model_quant_file, 'wb') as f:
    f.write(tflite_quant_model)

<class 'keras.src.engine.sequential.Sequential'>


INFO:tensorflow:Assets written to: /tmp/tmpyjmp876b/assets
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
